In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# importo los .py
import sys
sys.path.append("aux")
import config

from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
arch_entrada = "datasets/emp3_sellout_transf.csv"
arch_salida = "datasets/emp3_sellout_train.csv"

arch_productos = "datasets/emp3_sellout_producto_base.csv"
arch_clientes = "datasets/emp3_sellout_cliente_base.csv"
arch_cat1 = "datasets/emp3_sellout_cat1_base.csv"
arch_cat2 = "datasets/emp3_sellout_cat2_base.csv"
arch_cat3 = "datasets/emp3_sellout_cat3_base.csv"

In [4]:
def crear_nuevos_features(df_param):
    # Diferencia entre lo solicitado y lo entregado
    df_param["dif_cust_request_tn"]=df_param.cust_request_tn-df_param.tn
    df_param["dif_cust_request_tn_porc"]=np.where(df_param.cust_request_tn==0,0,100*df_param.dif_cust_request_tn/df_param.cust_request_tn)
   
    return df_param

In [5]:
def crear_features_temporales(campo,df_param):
    # Primero y FUNDAMENTAL, ordeno por el campo y periodo
    df_param = df_param.sort_values(by=[campo,"periodo"],ascending=True)

    if campo=="prod_cust":
        prefijo_tn = ""
    else:
        prefijo_tn = campo + "_"
    for i in range(1, config.NUM_LAGS_PARAM + 1):
        df_param[campo + "_tn_lag_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].shift(i)
        df_param[campo + "_tn_delta_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].diff(i)
        df_param[campo + "_rolling_std_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].rolling(window=i).std().reset_index(level=0, drop=True)
        df_param[campo + "_rolling_mean_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].rolling(i).mean().reset_index(level=0, drop=True)
        df_param[campo + "_rolling_sum_" + str(i)] = df_param.groupby(campo)[prefijo_tn + "tn"].rolling(window=i).sum().reset_index(level=0, drop=True)

    return df_param

In [6]:
df_sellout = pd.read_csv(arch_entrada)
df_productos = pd.read_csv(arch_productos)
df_clientes = pd.read_csv(arch_clientes)
df_cat1 = pd.read_csv(arch_cat1)
df_cat2 = pd.read_csv(arch_cat2)
df_cat3 = pd.read_csv(arch_cat3)

In [7]:
df_sellout = crear_nuevos_features(df_sellout)
df_sellout = crear_features_temporales("prod_cust",df_sellout)

df_productos = crear_features_temporales("product_id",df_productos)
df_clientes = crear_features_temporales("customer_id",df_clientes)
df_cat1 = crear_features_temporales("cat1",df_cat1)
df_cat2 = crear_features_temporales("cat2",df_cat2)
df_cat3 = crear_features_temporales("cat3",df_cat3)

print(len(df_sellout))
df_sellout = df_sellout.merge(df_productos,how="left",on=["product_id","periodo"])
df_sellout = df_sellout.merge(df_clientes,how="left",on=["customer_id","periodo"])
df_sellout = df_sellout.merge(df_cat1,how="left",on=["cat1","periodo"])
df_sellout = df_sellout.merge(df_cat2,how="left",on=["cat2","periodo"])
df_sellout = df_sellout.merge(df_cat3,how="left",on=["cat3","periodo"])

print(len(df_sellout))



4020
4020


In [8]:
df_sellout.head(100)

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,...,cat3_tn_lag_14,cat3_tn_delta_14,cat3_rolling_std_14,cat3_rolling_mean_14,cat3_rolling_sum_14,cat3_tn_lag_15,cat3_tn_delta_15,cat3_rolling_std_15,cat3_rolling_mean_15,cat3_rolling_sum_15
0,20001,201701,10001,0,0.0,11.0,0.277313,0.277313,2017-01-01,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201702,10001,0,0.0,23.0,0.554532,0.554532,2017-02-01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,201703,10001,0,0.0,33.0,0.257866,0.257866,2017-03-01,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,201704,10001,0,0.0,8.0,0.037083,0.037083,2017-04-01,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,201705,10001,0,0.0,15.0,0.282245,0.281683,2017-05-01,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20001,201812,10003,0,0.0,23.0,0.358943,0.358943,2018-12-01,12,...,5215.16209,-369.89866,822.238318,5175.651712,72459.12397,5299.00027,-453.73684,792.394340,5178.285737,77674.28606
96,20001,201901,10003,0,0.0,23.0,0.657724,0.655041,2019-01-01,1,...,4920.63610,-205.92089,828.965377,5160.943077,72253.20308,5215.16209,-500.44688,801.217126,5144.922612,77173.83918
97,20001,201902,10003,0,0.0,18.0,0.372967,0.294715,2019-02-01,2,...,4434.73833,368.62770,809.756306,5187.273627,72621.83078,4920.63610,-117.27007,804.128810,5137.104607,77056.56911
98,20001,201903,10003,0,0.0,18.0,0.786057,0.620163,2019-03-01,3,...,4549.79830,955.07698,791.955198,5255.493411,73576.90776,4434.73833,1070.13695,784.597926,5208.447071,78126.70606


In [9]:
df_sellout.to_csv(arch_salida, index=False)

## Verificaciones

In [10]:
# Nuevos features
df_control = df_sellout[df_sellout.dif_cust_request_tn_porc > 0]
df_control[["product_id","customer_id","periodo","tn","cust_request_tn","dif_cust_request_tn","dif_cust_request_tn_porc"]].sample(10)

,product_id,customer_id,periodo,tn,cust_request_tn,dif_cust_request_tn,dif_cust_request_tn_porc
621,20002,10006,201710,0.357709,0.394760,0.037051,9.385700
1768,20005,10002,201705,0.163719,0.169942,0.006223,3.661965
2227,20006,10002,201908,0.013181,0.162214,0.149033,91.874296
398,20001,10013,201703,0.441288,0.452270,0.010982,2.428256
129,20001,10004,201810,1.000000,1.087764,0.087764,8.068266
921,20003,10002,201810,1.000000,1.005248,0.005248,0.522023
465,20002,10001,201910,1.441677,1.486857,0.045180,3.038598
1605,20004,10009,201810,0.362500,0.370630,0.008130,2.193592
430,20001,10013,201911,0.540767,0.593525,0.052758,8.888891
506,20002,10003,201703,0.758530,0.758879,0.000348,0.045883


In [11]:
# Lags prod-cust
df_sellout[["prod_cust","periodo","tn","prod_cust_tn_lag_1","prod_cust_tn_lag_2","prod_cust_tn_lag_3"]].head(50)

,prod_cust,periodo,tn,prod_cust_tn_lag_1,prod_cust_tn_lag_2,prod_cust_tn_lag_3
0,20001-10001,201701,0.277313,NaN,NaN,NaN
1,20001-10001,201702,0.554532,0.277313,NaN,NaN
2,20001-10001,201703,0.257866,0.554532,0.277313,NaN
3,20001-10001,201704,0.037083,0.257866,0.554532,0.277313
4,20001-10001,201705,0.281683,0.037083,0.257866,0.554532
5,20001-10001,201706,0.357098,0.281683,0.037083,0.257866
6,20001-10001,201707,0.282245,0.357098,0.281683,0.037083
7,20001-10001,201708,0.120864,0.282245,0.357098,0.281683
8,20001-10001,201709,0.806935,0.120864,0.282245,0.357098
9,20001-10001,201710,0.619428,0.806935,0.120864,0.282245


In [12]:
# Delta Lags prod-cust
#df_sellout[["prod_cust","periodo","tn","tn_lag_1","tn_delta_1","tn_lag_2","tn_delta_2","tn_lag_3","tn_delta_3"]].head(50)

In [13]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4020 entries, 0 to 4019
Columns: 485 entries, product_id to cat3_rolling_sum_15
dtypes: bool(1), float64(470), int64(8), object(6)
memory usage: 14.9+ MB


In [14]:
df_sellout.isna().sum()

product_id                  0
periodo                     0
customer_id                 0
cero_ventas                 0
plan_precios_cuidados       0
                         ... 
cat3_tn_lag_15           1620
cat3_tn_delta_15         1620
cat3_rolling_std_15      1512
cat3_rolling_mean_15     1512
cat3_rolling_sum_15      1512
Length: 485, dtype: int64

In [15]:
df_sellout.columns

Index(['product_id', 'periodo', 'customer_id', 'cero_ventas',
       'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn',
       'periodo_fecha', 'mes',
       ...
       'cat3_tn_lag_14', 'cat3_tn_delta_14', 'cat3_rolling_std_14',
       'cat3_rolling_mean_14', 'cat3_rolling_sum_14', 'cat3_tn_lag_15',
       'cat3_tn_delta_15', 'cat3_rolling_std_15', 'cat3_rolling_mean_15',
       'cat3_rolling_sum_15'],
      dtype='object', length=485)

In [16]:
len(df_sellout.columns)

485